In [1]:
import networkx as nx

import pandas as pd

from random import sample

from collections import defaultdict

import matplotlib.pyplot as plt

In [8]:
def graph_value(vert, v, w, k = 0.5):
    s = 0
    for el in v:
        s += float(w[vert.index(el)][0])
    return s - k * len(v)

In [13]:
def best_clique(snps, midesp, vert_weight=1):
    dict1 = defaultdict(list)
    with open(snps,"r") as op:
        for line in op:
            line = line.strip()
            line = line.split()
            if '"CHR"' not in line:
                if float(line[7])>=0:#убираем отрицательные бета
                    dict1[line[2][1:-1]].append(line[7])
    with open(midesp,"r") as op2:
        dict2 = defaultdict(list)
        for line2 in op2:
            line2 = line2.strip()
            line2 = line2.split("+")
            line22 = line2[1].split(" ")
            if "SNP1 " not in line2:
                dict2[(line2[0].strip(),line22[1])].append(line22[2])
    dict3 = defaultdict(list)
    for key2 in dict2:
        key20 = key2[0].strip()
        key21 = key2[1].strip()
        if key20 in dict1.keys() and key21 in dict1.keys():
             if float(dict2[key2][0])>0:#порог на MI
                dict3[(key20,key21)].append(dict2[key2][0])
    vert = list(dict1.keys())
    beta = list(dict1.values())
    false_ed = []
    ed = list(dict3.keys())
    for i in range(len(ed)):
        false_ed.append([ed[i][0], ed[i][1]])
    graph = []
    for i in range(len(vert)):
        for j in range(i + 1, len(vert)):
            if [vert[i], vert[j]] not in false_ed and [vert[j], vert[i]]  not in false_ed:
                graph.append([vert[i], vert[j]])
    G = nx.Graph()
    for key2 in dict1:
        G.add_node(str(key2),  weight=float(dict1[key2][0]))
    for key in graph:
        G.add_edge(str(key[0]), str(key[1]))
        
    cliques = []
    for i in range(1000):
        for k in range(3, 10):
            subgraph = sample(list(G.nodes), k)
            try:
                a = list(nx.find_cliques(G, subgraph))
                cliques.append(subgraph)
            except ValueError:
                pass
    cl = []
    weights = []
    for clique in cliques:
        if type(clique[0]) == str:
            weights.append(graph_value(vert, clique, beta, 0.62))
            cl.append(clique)
        else:
             for i in clique:
                weights.append(graph_value(vert, i, beta, 0.62))
                cl.append(i)
    return cl[weights.index(max(weights))]


In [14]:
best_clique("significant_snps.txt", "soyabean_midesp_out.epi", 0.24)

['chr_19:37036197',
 'chr_18:2886428',
 'chr_19:37062741',
 'chr_19:36853376',
 'chr_18:2841589',
 'chr_19:34599708',
 'chr_19:36856526',
 'chr_13:21758530',
 'chr_02:5022182']